# SODA Toolchain

![tutorial-flow](imgs/flow-diagram.png)

# High-Level Application Input (TensorFlow)

### Build a model in tensorflow (Step 1)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
tf.random.set_seed(seed=0)
print(tf.__version__)

in1 = keras.layers.Input(shape=(32,32,1))
tmp = keras.layers.Conv2D(filters=1, kernel_size=(5,5),
                          input_shape=(32,32),
                          padding='same', 
                          strides=(2, 2),
                          activation='relu', 
                          use_bias=True)(in1)
tmp = keras.layers.Flatten()(tmp)
tmp = keras.layers.Dense(units=8, activation='relu')(tmp)
tmp = keras.layers.Dense(units=4, activation='relu')(tmp)
out = keras.layers.Dense(units=2, activation='softmax')(tmp)
model = keras.models.Model(inputs=[in1], outputs=out)

# Compile model with optimizer
model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

2.4.1


2022-03-21 03:47:53.383003: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-21 03:47:53.383311: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 03:47:53.386788: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Convert model to protobuf

In [2]:
!mkdir -p output

In [3]:
save_path = os.path.join(os.getcwd(), "model/simple/")

# Save model to SavedModel format
tf.saved_model.save(model, save_path)

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=[
        tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name='x1')
    ])

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                    logdir=os.getcwd(),
                    name="output/frozen_graph.pbtxt",
                    as_text=True)


2022-03-21 03:48:05.765390: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/model/simple/assets


2022-03-21 03:48:06.073309: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-21 03:48:06.073522: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-21 03:48:06.093406: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2795020000 Hz
2022-03-21 03:48:06.096859: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.005ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.



'/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/frozen_graph.pbtxt'

![tutorial-flow](imgs/flow-diagram.png)

### Transform protobuf into MLIR (Step 2)




In [4]:
!scripts/protobuf-to-tosa.sh output/frozen_graph.pbtxt output/tosa.mlir

2022-03-21 10:48:31.718652: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Lower MLIR to Linalg on Buffers (Step 3)

In [5]:
!scripts/tosa-to-linalg.sh output/tosa.mlir output/linalg-buffers.mlir

![tutorial-flow](imgs/flow-diagram.png)

# SODA-OPT: HW/SW Partitioning and Optimizer (Step 4)

## How to use soda.launch?

### Automatic selection of custom accelerator region

Using the pass: `-convert-<abstraction_name>-<operation_name>-to-soda`

Such as: `-convert-linalg-generic-to-soda`

### Manual selection of custom accelerator region

Adding the following lines around any code that will become the accelerator:

```
soda.launch {
  // ...
  // Code to be transformed into an accelerator
  // ...
  soda.terminator
}
```

Run next cell and edit [file](output/01searched-edited.mlir).

In [6]:
!cp output/linalg-buffers.mlir output/01searched-edited.mlir

# Perform manual edit!

Edit the [file](output/01searched-edited.mlir).

## Optimization pipeline

![optimizations](imgs/optimization-table.png)

### Kernel without SODA-OPT optimizations (Baseline)

In [7]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode=no-aa \
    -lower-all-to-llvm=use-bare-ptr-memref-call-conv \
    -print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04baseline.mlir \
    2>&1 | cat > output/05intermediate-baseline.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate \
    --mlir-to-llvmir \
    output/04baseline.mlir \
    -o output/05baseline.ll
)

Visualize [intermediate file](output/05intermediate-baseline.mlir)

### Kernel with SODA-OPT optimizations

In [8]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu=use-bare-ptr-memref-call-conv \
    -print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)


Visualize [intermediate file](output/05intermediate-optimized.mlir)

![tutorial-flow](imgs/flow-diagram.png)

# Bambu: Synthesizing the Outlined Kernel (Step 5)

The following configurations are passed to our backend HLS tool:

* Target: ASIC generation using the Nangate cell library with the FreePDK 45nm kit
* Memory technology: SRAM
* Number of memory channels: 2
  * Supports 2 parallel reads and 2 parallel writes
* Target frequency: 200MHz (5ns period)
* Using bambu's floating-point operation support

### Baseline kernel

In [9]:
! scripts/run-bambu.sh baseline

/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/baseline /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG10 -O2 --device=nangate45 --clock-period=5 --no-iob --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_test.xml --simulate --simulator=VERILATOR --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                

In [10]:
baseline_runtime = ""

for runtime in open('output/baseline/bambu-log').readlines():
    if "Average execution" in runtime:
        baseline_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(baseline_runtime))

Average execution in cycles: 1158


Visualize [Intermediate Dot File](output/baseline/HLS_output/dot/main_kernel/HLS_STGraph.dot)

### Optimized kernel

In [11]:
! scripts/run-bambu.sh optimized
# Takes aprox 30 seconds to execute

/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/optimized /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG10 -O2 --device=nangate45 --clock-period=5 --no-iob --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_test.xml --simulate --simulator=VERILATOR --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
               

In [12]:
optimized_runtime = ""

for runtime in open('output/optimized/bambu-log').readlines():
    if "Average execution" in runtime:
        optimized_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(optimized_runtime))


Average execution in cycles: 55


Visualize [Intermediate Dot File](output/optimized/HLS_output/dot/main_kernel/HLS_STGraph.dot)

## Comparison of runtime results

* Display runtime
* Display [verilog output file](output/optimized/main_kernel.v)

In [13]:
print("Average execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Average execution in cycles of Optimized kernel: {}".format(optimized_runtime))
print("Speedup: {:.1f}".format(float(baseline_runtime/optimized_runtime)))

Average execution in cycles of Baseline kernel:  1158
Average execution in cycles of Optimized kernel: 55
Speedup: 21.1


# Commandline interface

To visualize all possible paramenters for our optimization passes run:

- `soda-opt -h`

```
      --soda-opt-pipeline-for-bambu                    
        --affine-tile-size=<ulong>                     
        --bitwidth-of-index-type=<uint>                
        --max-alloc-size-in-bytes=<uint>               
        --max-rank-of-allocated-memref=<uint>          
        --number-of-full-unrolls=<uint>                
        --permutation-map=<uint>                       
        --use-bare-ptr-memref-call-conv                
        --no-alloca-promotion                          
        --no-buffer-trick                              
        --no-scalar-replacement                        
  
```

In [14]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt -h 2>&1 | cat > output/helpfile
)

Open [help file](output/helpfile)

### Modifying the number of unrolls

In [15]:
%%bash
(
  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-xml=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv number-of-full-unrolls=1" \
    -print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  docker run -u $(id -u) -v $(pwd):/working_dir --rm agostini01/soda \
  mlir-translate \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)

Visualize [intermediate file](output/05intermediate-optimized.mlir)

In [16]:
! scripts/run-bambu.sh optimized

/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/optimized /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG10 -O2 --device=nangate45 --clock-period=5 --no-iob --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_test.xml --simulate --simulator=VERILATOR --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
               

![tutorial-flow](imgs/flow-diagram.png)

# OpenRoad Flow: Automatic ASIC place and route (Step 6)

### Baseline kernel

In [17]:
! scripts/run-openroad.sh baseline

# Approx. 4min to execute

/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/baseline /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version
[INFO][FLOW] Using platform directory /OpenROAD-flow-scripts/flow/platforms/nangate45
/OpenROAD-flow-scripts/flow/util/markDontUse.py -p "TAPCELL_X1 FILLCELL_X1 AOI211_X1 OAI211_X1" -i /OpenROAD-flow-scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib -o /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
Opening file for replace: /OpenROAD-flow-scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib
Marked 4 cells as dont_use
Commented 0 lines containing "original_pin"
Writing replaced file: /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
mkdir -p /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/results/n

### Optimized kernel

In [18]:
! scripts/run-openroad.sh optimized

# Approx. 23min to execute

/files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version/output/optimized /files0/extended/bohm747/Development/soda/soda-opt/docs/tutorials/date2022/docker-version
[INFO][FLOW] Using platform directory /OpenROAD-flow-scripts/flow/platforms/nangate45
/OpenROAD-flow-scripts/flow/util/markDontUse.py -p "TAPCELL_X1 FILLCELL_X1 AOI211_X1 OAI211_X1" -i /OpenROAD-flow-scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib -o /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
Opening file for replace: /OpenROAD-flow-scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib
Marked 4 cells as dont_use
Commented 0 lines containing "original_pin"
Writing replaced file: /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
mkdir -p /Panda-flow/HLS_output/Synthesis/bash_flow/openroad/results/

## Comparison of synthesis results

* Display area
* Display power
* Calculate and display FLOPS/W

In [19]:
log_path_suffix='HLS_output/Synthesis/bash_flow/openroad/logs/nangate45/main_kernel/base/6_report.log'
gds_path_suffix='HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds'

### Baseline

In [20]:
baseline_log='output/baseline/'+log_path_suffix

In [21]:
power_multiplier = 1 # Open road reports power in mW

log_file=baseline_log
total_power = ()

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Baseline accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))


baseline_total_power=total_power
baseline_available_area=available_area
baseline_utilization_area=utilization_area


Baseline accelerator:
  total power consumption: 0.00642W
  available chip area: 28904.0 um^2
  utilized chip area: 2.0%


### Optimized for runtime

In [22]:
optimized_log='output/optimized/'+log_path_suffix

In [23]:
log_file=optimized_log
total_power = ()

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Optimized accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))

optimized_total_power=total_power
optimized_available_area=available_area
optimized_utilization_area=utilization_area

Optimized accelerator:
  total power consumption: 0.0617W
  available chip area: 238968.0 um^2
  utilized chip area: 16.0%


## Post place and route comparison

Considering a matrix multiply kernel has approximatelly 2xNxMxK arithmetic operations

And our selected kernel has the following sizes: 

```
linalg.batch_matmul ins(%23, %6 : memref<1x4x8xf32>, memref<1x8x4xf32>) 
                    outs(%25 : memref<1x4x4xf32>)
```
M=4, K=8, N=4

We have approximatelly **256** floating point aritihmetic operations

In [24]:
giga_multiplier=1e9
flop_count = 256 # arithmetic float point operations
target_frequency = 200e+6 # 200MHz

optimized_runtime_in_s = optimized_runtime/target_frequency
baseline_runtime_in_s = baseline_runtime/target_frequency 

baseline_flops_per_watt= flop_count/baseline_runtime_in_s/baseline_total_power
optimized_flops_per_watt= flop_count/optimized_runtime_in_s/optimized_total_power


print("Execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Execution in cycles of Optimized kernel:   {}".format(optimized_runtime))

print("Speedup: \t\t\t{:.2f}x".format(baseline_runtime/optimized_runtime))
print("Area utilization overhead: \t {:.2f}x".format(optimized_utilization_area/baseline_utilization_area))
print("Power overhead: \t\t {:.2f}x".format(optimized_total_power/baseline_total_power))

print("Baseline  \t\t\t {:.2f} GFLOPS/W ".format(baseline_flops_per_watt/giga_multiplier))
print("Optimized \t\t\t{:.2f} GFLOPS/W".format(optimized_flops_per_watt/giga_multiplier))


Execution in cycles of Baseline kernel:  1158
Execution in cycles of Optimized kernel:   55
Speedup: 			21.05x
Area utilization overhead: 	 8.00x
Power overhead: 		 9.61x
Baseline  			 6.89 GFLOPS/W 
Optimized 			15.09 GFLOPS/W


## Generated GDSII files

Output files can be found here:

* output/baseline/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds
* output/optimized/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds

### Baseline and Optimized Side by Side

<p float="middle">
  <img src="imgs/baseline_view.png" width="49%" />
  <img src="imgs/optimized_view.png" width="49%" /> 
</p>

# Thank you!